# Async IO from Ground Up

Understanding Python async features from the first principles.

## What is Async

Lets compare synchronous and asyncronous styles.

### Synchronous - blocking

[cartoon will come here]

- X visits Bangalore Iyengar Bakery
- Orders a veg puff
- waits until it is ready
- takes it and walks away

### Asynchronous - callback

[Cartoon will come here]

- X visits Starbucks
- Orders black coffee
- Sits and reads about asyncio
- The bartista calls his name and gives his coffee

### Async - callback style programming

```javascript
// node.js application to download a URL
var http = require('http');

function wget(url, callback) {
    http.get(url, function (response) {
        var data = '';
        response.setEncoding('utf8');
        
        response.on('data', function(chunk) {
            data += chunk;
        }).on('end', function() {
            callback(data);
        });
    });
}
```    

### Async - python

In [1]:
import asyncio
import aiohttp

@asyncio.coroutine
def wget(url):
    with aiohttp.ClientSession() as client:
        response = yield from client.get(url)
        data = yield from response.text()
        return data

Python generators makes it possible to write asynchronous programs without having to deal with the complexity of callbacks.

Python 2.5 added a two new constructs `async` and `await` to make it even simpler to write async code.

In [2]:
async def wget(url):
    with aiohttp.ClientSession() as client:
        response = await client.get(url)
        data = await response.text()
        return data

## How does this work? 

Lets start from the beginning.

* Generators
* Coroutines
* Event loops
* `async` and `await`

## Generators

## Coroutines

We've seen that generators are good at modelling various kinds of data flow, but it is also possible to use generators to deal with control flow.

Lets try an experiment to run multiple generators concurrently.

In [3]:
def interweave(generators):
    while True:
        for g in generators:
            yield next(g)

def run_all(generators):
    for x in interweave(generators):
        pass

In [4]:
def squares(numbers):
    for n in numbers:
        print("Computing square of", n)
        yield n*n

In [5]:
g1 = squares([1, 2, 3, 4])
g2 = squares([10, 20, 30, 40])

In [6]:
run_all([g1, g2])

Computing square of 1
Computing square of 10
Computing square of 2
Computing square of 20
Computing square of 3
Computing square of 30
Computing square of 4
Computing square of 40


This implementation of `interweave` is very primitive. It expects all the generators to be of same length. That is not hard to fix.

In [9]:
from collections import deque

def interweave(generators):
    # create a queue of generators
    generators = deque(generators)
    while generators:
        # pop and progress the left most one.
        g = generators.popleft()
        try:
            yield next(g)
        except StopIteration:
            pass
        else:
            # if it has more elements, enqueue it again.
            generators.append(g)

In [10]:
g1 = squares([1, 2, 3, 4, 5, 6])
g2 = squares([10, 20, 30, 40])
run_all([g1, g2])

Computing square of 1
Computing square of 10
Computing square of 2
Computing square of 20
Computing square of 3
Computing square of 30
Computing square of 4
Computing square of 40
Computing square of 5
Computing square of 6


In [13]:
def square(x):
    yield
    print("Computing square of ", x)
    return x*x

def sum_of_squares(numbers):
    result = 0
    for n in numbers:
        n2 = yield from square(n)
        result += n2
    yield
    print("sum_of_squares", numbers, result)
    return result

In [14]:
g1 = sum_of_squares([1, 2, 3, 4, 5])
g2 = sum_of_squares([10, 20, 30, 40, 50])
run_all([g1, g2])

Computing square of  1
Computing square of  10
Computing square of  2
Computing square of  20
Computing square of  3
Computing square of  30
Computing square of  4
Computing square of  40
Computing square of  5
Computing square of  50
sum_of_squares [1, 2, 3, 4, 5] 55
sum_of_squares [10, 20, 30, 40, 50] 5500
